In [1]:
from diquark.data.loader import DataLoader
from diquark.features.feature_extractor import FeatureExtractor

from diquark.config.constants import PATH_DICT_ATLAS_136_80

In [2]:
loader = DataLoader(PATH_DICT_ATLAS_136_80, index_stop=1000)

In [3]:
datasets = loader.load_data()

Loading data: 100%|██████████| 30/30 [00:05<00:00,  5.21it/s]


In [4]:
all_features = {}
for key, data in datasets.items():
    extractor = FeatureExtractor(n_jets=6)
    all_features[key] = extractor.compute_all(data)

/home/idinu/local/diquark-analysis/.venv/lib/python3.11/site-packages/awkward/_nplikes/array_module.py:249: RuntimeWarning: invalid value encountered in sqrt
  return impl(*broadcasted_args, **(kwargs or {}))
/home/idinu/local/diquark-analysis/diquark/features/feature_extractor.py:172: RuntimeWarning: Mean of empty slice.
  m3j.mean(axis=1, where=m3j != 0),
/home/idinu/local/diquark-analysis/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/home/idinu/local/diquark-analysis/diquark/features/feature_extractor.py:179: RuntimeWarning: Mean of empty slice.
  m2j.mean(axis=1, where=m2j != 0),


In [5]:
for k, v in all_features["SIG:Suu"].items():
    print(k, v.ndim, v.shape)
    print("")

jet_multiplicity 1 (1000,)

leading_jet_pt_1 1 (1000,)

leading_jet_pt_2 1 (1000,)

leading_jet_pt_3 1 (1000,)

leading_jet_pt_4 1 (1000,)

leading_jet_pt_5 1 (1000,)

leading_jet_pt_6 1 (1000,)

leading_jet_eta_1 1 (1000,)

leading_jet_eta_2 1 (1000,)

leading_jet_eta_3 1 (1000,)

leading_jet_eta_4 1 (1000,)

leading_jet_eta_5 1 (1000,)

leading_jet_eta_6 1 (1000,)

leading_jet_phi_1 1 (1000,)

leading_jet_phi_2 1 (1000,)

leading_jet_phi_3 1 (1000,)

leading_jet_phi_4 1 (1000,)

leading_jet_phi_5 1 (1000,)

leading_jet_phi_6 1 (1000,)

delta_r_1 1 (1000,)

delta_r_2 1 (1000,)

delta_r_3 1 (1000,)

delta_r_4 1 (1000,)

delta_r_5 1 (1000,)

delta_r_6 1 (1000,)

delta_r_7 1 (1000,)

delta_r_8 1 (1000,)

delta_r_9 1 (1000,)

delta_r_10 1 (1000,)

delta_r_11 1 (1000,)

delta_r_12 1 (1000,)

delta_r_13 1 (1000,)

delta_r_14 1 (1000,)

delta_r_15 1 (1000,)

combined_invariant_mass 1 (1000,)

2jet_invariant_mass_1 1 (1000,)

2jet_invariant_mass_2 1 (1000,)

2jet_invariant_mass_3 1 (1000,)

2

In [6]:
import yaml
from diquark.data.loader import DataLoader
from diquark.features.feature_extractor import FeatureExtractor
from diquark.data.preprocessor import Preprocessor
from diquark.models.gradient_boosting import GradientBoostingModel
from diquark.models.neural_network import NeuralNetworkModel
from diquark.config.constants import PATH_DICT_ATLAS_136_80

# Load configuration
with open("../diquark/config/default_settings.yaml", "r") as file:
    config = yaml.safe_load(file)

# Load data
loader = DataLoader(PATH_DICT_ATLAS_136_80, index_stop=config["data"]["index_stop"])
datasets = loader.load_data()

# Extract features
all_features = {}
for key, data in datasets.items():
    extractor = FeatureExtractor(n_jets=config["feature_extraction"]["n_jets"])
    all_features[key] = extractor.compute_all(data)


/home/idinu/local/diquark-analysis/.venv/lib/python3.11/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
2024-11-11 23:59:02.991345: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-11 23:59:02.992186: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] 

In [7]:
with open("../diquark/config/default_settings.yaml", "r") as file:
    config = yaml.safe_load(file)

# Preprocess data
preprocessor = Preprocessor(config["preprocessing"])
X_train, X_test, y_train, y_test, df_train, df_test = preprocessor.prepare_data(all_features)

model = NeuralNetworkModel(config["models"]["neural_network"])
model.build(input_shape=X_train.shape[1])
training_results = model.train(X_train, y_train, X_test, y_test)

# Make predictions
y_pred = model.predict(X_test)

# Print some results
print("Training Results:", training_results)
# print("Feature Importances:", model.feature_importances())

# You can add more evaluation metrics here, such as accuracy, AUC, etc.
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

print("AUC:", roc_auc_score(y_test, y_pred))
precision, recall, _ = precision_recall_curve(y_test, y_pred)
print("AUC-PR:", auc(recall, precision))

Epoch 1/5


/home/idinu/local/diquark-analysis/.venv/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


725/725 ━━━━━━━━━━━━━━━━━━━━ 1s 913us/step - accuracy: 0.4962 - loss: 0.6937 - val_accuracy: 0.0333 - val_loss: 0.7132
Epoch 2/5
725/725 ━━━━━━━━━━━━━━━━━━━━ 1s 853us/step - accuracy: 0.5020 - loss: 0.6932 - val_accuracy: 0.9667 - val_loss: 0.6704
Epoch 3/5
725/725 ━━━━━━━━━━━━━━━━━━━━ 1s 856us/step - accuracy: 0.5006 - loss: 0.6932 - val_accuracy: 0.0333 - val_loss: 0.7009
Epoch 4/5
725/725 ━━━━━━━━━━━━━━━━━━━━ 1s 849us/step - accuracy: 0.5001 - loss: 0.6933 - val_accuracy: 0.0333 - val_loss: 0.7282
Epoch 5/5
725/725 ━━━━━━━━━━━━━━━━━━━━ 1s 854us/step - accuracy: 0.5017 - loss: 0.6933 - val_accuracy: 0.9667 - val_loss: 0.6796
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 341us/step
Training Results: <keras.src.callbacks.history.History object at 0x7f9e1454e950>
AUC: 0.5
AUC-PR: 0.5166666666666667


In [9]:
model.model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         4,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,293 (94.90 KB)

 Trainable params: 8,097 (31.63 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 16,196 (63.27 KB)

In [10]:
model = GradientBoostingModel(config["models"]["gradient_boosting"])
model.build(input_shape=X_train.shape[1])
training_results = model.train(X_train, y_train, X_test, y_test)

# Make predictions
y_pred = model.predict(X_test)

# Print some results
print("Training Results:", training_results)
print("Feature Importances:", model.feature_importances())

# You can add more evaluation metrics here, such as accuracy, AUC, etc.
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

print("AUC:", roc_auc_score(y_test, y_pred))
precision, recall, _ = precision_recall_curve(y_test, y_pred)
print("AUC-PR:", auc(recall, precision))

[0]	validation_0-logloss:0.60807
[1]	validation_0-logloss:0.53853
[1]	validation_0-logloss:0.53853
[2]	validation_0-logloss:0.48069
[3]	validation_0-logloss:0.43176
[4]	validation_0-logloss:0.39023
[5]	validation_0-logloss:0.35460
[6]	validation_0-logloss:0.32350
[7]	validation_0-logloss:0.29679
[8]	validation_0-logloss:0.27355
[9]	validation_0-logloss:0.25289
[10]	validation_0-logloss:0.23456
[11]	validation_0-logloss:0.21854
[12]	validation_0-logloss:0.20456
[13]	validation_0-logloss:0.19182
[14]	validation_0-logloss:0.18056
[15]	validation_0-logloss:0.17042
[16]	validation_0-logloss:0.16154
[17]	validation_0-logloss:0.15396
[18]	validation_0-logloss:0.14664
[19]	validation_0-logloss:0.14030
[20]	validation_0-logloss:0.13482
[21]	validation_0-logloss:0.12938
[22]	validation_0-logloss:0.12445
[23]	validation_0-logloss:0.12039
[24]	validation_0-logloss:0.11653
[25]	validation_0-logloss:0.11304
[26]	validation_0-logloss:0.10972
[27]	validation_0-logloss:0.10655
[28]	validation_0-logloss

# Jet Analysis

In [11]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go


# Function to extract values and errors from a string like "1.1681e-01 ± 1.0086e-02"
def extract_value_and_error(s):
    value, error = map(float, s.split("±"))
    return value, error

In [41]:
df_6j = pd.read_csv("../jet_no_analysis/ATLAS_136_70_6j_5f/random_forest_counts_summary.csv")
df_7j = pd.read_csv("../jet_no_analysis/ATLAS_136_70_7j_5f/random_forest_counts_summary.csv")
df_8j = pd.read_csv("../jet_no_analysis/ATLAS_136_70_8j_5f/random_forest_counts_summary.csv")

In [42]:
# Combine dataframes
dfs = [df_6j, df_7j, df_8j]
labels = ["6 jets", "7 jets", "8 jets"]

thresholds = [float(col) for col in df_6j.columns if col != "Process"][2:-1]
colors = ["rgb(31, 119, 180)", "rgb(255, 127, 14)", "rgb(44, 160, 44)"]

In [44]:
# Create figure
fig = go.Figure()

for df, label, color in zip(dfs, labels, colors):
    # Extract S/B values and errors
    sb_row = df[df["Process"] == "S/B"].iloc[0]
    sb_values = []
    sb_errors = []
    for threshold in thresholds:
        value, error = extract_value_and_error(sb_row[str(threshold)])
        sb_values.append(value)
        sb_errors.append(error)

    # Convert to numpy arrays for easier calculations
    sb_values = np.array(sb_values)
    sb_errors = np.array(sb_errors)

    # Add trace for the central S/B values
    fig.add_trace(
        go.Scatter(
            x=thresholds, y=sb_values, mode="lines", name=f"S/B ({label})", line=dict(color=color)
        )
    )

    # Add error bands
    fig.add_trace(
        go.Scatter(
            x=thresholds + thresholds[::-1],
            y=np.concatenate([sb_values + sb_errors, (sb_values - sb_errors)[::-1]]),
            fill="toself",
            fillcolor=color.replace("rgb", "rgba").replace(")", ",0.2)"),  # Add transparency
            line=dict(color="rgba(255,255,255,0)"),
            hoverinfo="skip",
            name=f"Error band ({label})",
        )
    )

# Update layout
fig.update_layout(
    title="Signal to Background Ratio vs Threshold",
    xaxis_title="Threshold",
    yaxis_title="S/B Ratio",
    yaxis_type="log",
    legend_title="Legend",
    xaxis=dict(tickmode="array", tickvals=thresholds, ticktext=[f"{t:.3f}" for t in thresholds]),
    width=800,
    height=600,
)

# Show the plot
fig.show()

In [15]:
df_6j

,0.2,0.5,0.8,0.9,0.925,0.95,0.96,0.97,0.98,0.99,Process
0,6.18223e-05 ± 7.65271e-07,6.18223e-05 ± 7.65271e-07,6.18223e-05 ± 7.65271e-07,4.332e-05 ± 1.078e-05,3.03408e-05 ± 9.62135e-07,1.64236e-05 ± 8.24784e-07,9.3763e-06 ± 3.6185e-07,3.4632e-06 ± 1.3341e-07,6.4629e-07 ± 5.7994e-08,5.584e-08 ± 3.561e-08,BKG:ff_HW
1,5.11999e-05 ± 8.57546e-07,5.11999e-05 ± 8.57546e-07,5.11999e-05 ± 8.57546e-07,3.5462e-05 ± 9.6213e-06,2.50293e-05 ± 9.02187e-07,1.35913e-05 ± 5.26367e-07,8.0364e-06 ± 2.7674e-07,2.7577e-06 ± 2.4684e-07,5.8677e-07 ± 8.8268e-08,1.3110e-07 ± 5.1391e-08,BKG:ff_HZ
2,3.0367e+00 ± 3.0158e-01,3.0367e+00 ± 3.0158e-01,3.0367e+00 ± 3.0158e-01,1.012e+00 ± 1.189e+00,2.1385e-01 ± 4.5160e-02,7.90e-03 ± 1.08e-02,3.936e-03 ± 8.800e-03,0.000e+00 ± 0.000e+00,0.000e+00 ± 0.000e+00,0.000e+00 ± 0.000e+00,BKG:ff_Hff
3,4.1429e+00 ± 6.1893e-01,4.1429e+00 ± 6.1893e-01,4.1429e+00 ± 6.1893e-01,1.252e+00 ± 1.385e+00,2.130e-01 ± 1.412e-01,2.377e-02 ± 3.555e-02,7.96e-03 ± 1.78e-02,0.000e+00 ± 0.000e+00,0.000e+00 ± 0.000e+00,0.000e+00 ± 0.000e+00,BKG:ff_Hff2
4,1.970986e-01 ± 6.003548e-04,1.970986e-01 ± 6.003548e-04,1.970986e-01 ± 6.003548e-04,1.6280e-01 ± 1.9475e-02,1.32821e-01 ± 1.20513e-03,8.2055e-02 ± 1.4505e-03,5.1228e-02 ± 1.2449e-03,1.9540e-02 ± 1.2693e-03,4.0918e-03 ± 4.4160e-04,7.462e-04 ± 2.608e-04,BKG:ff_WW_bkg
5,7.1573e-03 ± 5.8605e-04,7.1573e-03 ± 5.8605e-04,7.1573e-03 ± 5.8605e-04,4.201e-03 ± 1.496e-03,2.5029e-03 ± 3.2091e-04,9.600e-04 ± 1.812e-04,4.409e-04 ± 1.123e-04,1.0186e-04 ± 2.7550e-05,6.765e-06 ± 9.264e-06,0.000e+00 ± 0.000e+00,BKG:ff_ZW_bkg
6,5.56318e-03 ± 4.54129e-05,5.56318e-03 ± 4.54129e-05,5.56318e-03 ± 4.54129e-05,4.1438e-03 ± 7.8611e-04,3.05978e-03 ± 4.80027e-05,1.70375e-03 ± 3.83726e-05,1.02782e-03 ± 5.94553e-05,3.5783e-04 ± 1.6189e-05,5.500e-05 ± 1.206e-05,5.930e-06 ± 4.840e-06,BKG:ff_gmZgmZ_bkg
7,2.0657e-02 ± 1.0871e-03,2.0657e-02 ± 1.0871e-03,2.0657e-02 ± 1.0871e-03,1.6272e-02 ± 3.2268e-03,1.26249e-02 ± 6.36074e-04,7.1692e-03 ± 4.6861e-04,4.3379e-03 ± 3.5819e-04,1.4328e-03 ± 2.7065e-04,3.564e-04 ± 1.093e-04,1.0761e-04 ± 2.2528e-05,BKG:ffbar_Wgm
8,1.3404e-02 ± 4.0601e-03,1.3404e-02 ± 4.0601e-03,1.3404e-02 ± 4.0601e-03,7.303e-03 ± 2.487e-03,4.199e-03 ± 1.895e-03,8.114e-04 ± 5.656e-04,4.049e-04 ± 3.696e-04,0.000e+00 ± 0.000e+00,0.000e+00 ± 0.000e+00,0.000e+00 ± 0.000e+00,BKG:gg_Hg
9,2.01615e-03 ± 2.94173e-05,2.01615e-03 ± 2.94173e-05,2.01615e-03 ± 2.94173e-05,1.4853e-03 ± 2.6998e-04,1.06020e-03 ± 1.80758e-05,4.89793e-04 ± 9.11248e-06,2.35847e-04 ± 4.79702e-06,4.5399e-05 ± 3.1059e-06,3.471e-06 ± 1.054e-06,2.937e-07 ± 3.597e-07,BKG:gg_Httbar
